In [1]:
import sys
import os

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src')) 

import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
os.path.dirname(os.getcwd())

'/Users/samuelluxenberg/Library/CloudStorage/OneDrive-Personal/GitHub/end-to-end-mlb-betting'

In [3]:
sys.path

['/opt/homebrew/Cellar/python@3.10/3.10.17_1/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.17_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.17_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/samuelluxenberg/Library/CloudStorage/OneDrive-Personal/GitHub/end-to-end-mlb-betting/.venv/lib/python3.10/site-packages',
 '/Users/samuelluxenberg/Library/CloudStorage/OneDrive-Personal/GitHub/end-to-end-mlb-betting/src']

In [4]:
from features.load_games_from_db import load_games_from_db

In [5]:
df = load_games_from_db()

2025-07-30 22:02:32,989 [INFO] Successully loaded 11041 games from database


Step 1: Empty DataFrame
Columns: [game_id, game_date, game_date_time, home_team_id, away_team_id, home_team, away_team, home_score, away_score, state, venue, game_type]
Index: []
Step 2: Empty DataFrame
Columns: [game_id, game_date, game_date_time, home_team_id, away_team_id, home_team, away_team, home_score, away_score, state, venue, game_type]
Index: []
There are 11041 games for  modeling...


In [6]:
df['game_date'] = pd.to_datetime(df['game_date'])

df.loc[df['home_team']=='Cleveland Indians', 'home_team'] = 'Cleveland Guardians'
df.loc[df['away_team']=='Cleveland Indians', 'away_team'] = 'Cleveland Guardians'

df.loc[df['home_team']=='Athletics', 'home_team'] = 'Oakland Athletics'
df.loc[df['away_team']=='Athletics', 'away_team'] = 'Oakland Athletics'

In [11]:
def team_schedule(
    df: pd.DataFrame, 
    date_col: str = 'game_date', 
    date_time_col: str = 'game_date_time'
) -> pd.DataFrame:
     
    """
    Transform a games DataFrame into a team-centric schedule view.
    
    Takes a DataFrame with game data (home_team vs away_team format) and converts it
    into a schedule where each row represents one team's game, with an indicator
    for whether they played at home or away.
    
    Args:
        df (pd.DataFrame): DataFrame containing game data with columns:
            - 'home_team': Name of the home team
            - 'away_team': Name of the away team
            - date_col: Column name containing game dates
            - date_time_col: Column name containing game datetime stamps
        date_col (str, optional): Name of the date column. Defaults to 'game_date'.
        date_time_col (str, optional): Name of the datetime column. Defaults to 'game_date_time'.
    
    Returns:
        pd.DataFrame: Team schedule DataFrame with columns:
            - 'team': Team name
            - date_col: Game date (same as input)
            - date_time_col: Game datetime (same as input)
            - 'home_ind': Binary indicator (1 if home game, 0 if away game)
            
        The DataFrame is sorted by team name and then by game datetime.
     """
    # Extract home and away games for every team
    cols = ['home_team','away_team', date_col, date_time_col, 'home_score','away_score']
    home_schedule = (
        df[cols]
        .rename(columns={
            'home_team': 'team',
            'away_team': 'opp_team',
            'home_score': 'team_score',
            'away_score': 'opp_score'
        })
        .assign(home_ind=1)
    )
    
    away_schedule = (
        df[cols]
        .rename(columns={
            'away_team': 'team',
            'home_team': 'opp_team',
            'home_score': 'opp_score',
            'away_score': 'team_score'
        })
        .assign(home_ind=0)
    )

    # Join them into one 'team' column
    team_schedule = (
        pd.concat([home_schedule, away_schedule])
        .sort_values(['team',date_time_col])
        .reset_index(drop=True)
    )

    # Create team win column
    team_schedule['team_win'] = (team_schedule['team_score'] > team_schedule['opp_score']).astype(int)

    # Create run differential column (team_score - opp_score)
    team_schedule['team_run_diff'] = team_schedule['team_score'] - team_schedule['opp_score']

    # Create season variable (for grouping)
    team_schedule['season'] = team_schedule['game_date'].dt.year.astype('int64')

    return team_schedule

def team_rest_days(
        df: pd.DataFrame,
        date_time_col: str = 'game_date_time',
        date_col: str = 'game_date'
        ) -> pd.Series:
    """
    Calculate the number of rest days between consecutive games for each team.
    
    This function sorts games chronologically and calculates how many days of rest
    each team had before each game by finding the difference between consecutive
    game dates for each team.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing game data with team and date information.
    team : {'home', 'away'}
        Specifies whether to calculate rest days for home teams or away teams.
        Will look for corresponding '{team}_team' column in the DataFrame.
    date_time_col : str, default 'game_date_time'
        Column name containing datetime information used for chronological sorting. 
        Necessary for sorting between double headers for a single team.
    date_col : str, default 'game_date'
        Column name containing date information used to calculate day differences.
        
    Returns
    -------
    pd.Series
        Series with the same index as input DataFrame containing the number of
        rest days for each game. First game for each team will have 0 rest days.
    """
    df_team_sched = team_schedule(df, date_time_col=date_time_col, date_col=date_col)
    return (
        df_team_sched
        .groupby(['team', 'season'])[date_col]
        .diff()
        .dt.days
        .fillna(value=0)
        -1
    ).clip(lower=0)

def team_games_previous_7days(
        df: pd.DataFrame, 
        date_col: str = 'game_date',
        date_time_col: str = 'game_date_time'
        ) -> pd.Series:
    """
    Calculate the 7-day rolling average of rest days for each team.
    
    This function computes a rolling average of rest days over the previous 7 games
    for each team, providing a smoothed measure of how well-rested teams have been
    over recent games. Uses transform to maintain the original DataFrame index.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing game data with team information and rest days.
        Must include a column named '{team}_rest_days' (e.g., 'home_rest_days').
    team : {'home', 'away'}
        Specifies whether to calculate rolling average for home or away teams.
        Will look for '{team}_team' and '{team}_rest_days' columns.
    date_col : str, default 'game_date'
        Column name containing date information used for chronological sorting.
        
    Returns
    -------
    pd.Series
        Series with the same index as input DataFrame containing the 7-day
        rolling average of rest days for each team and game.
    """
    df_team_sched = team_schedule(df, date_time_col=date_time_col, date_col=date_col)
    
    return (
        df_team_sched
        .groupby(['team'])
        .rolling(window='7D', on=date_col, min_periods=1, closed='left')
        .count()['game_date_time']
        .fillna(value=0)
        .reset_index(drop=True,level=0)
    )

def team_matchup_features(
    df: pd.DataFrame,
    date_col: str = 'game_date',
    date_time_col: str = 'game_date_time'
) -> pd.DataFrame:
    """
    Calculates several matchup-level features including: 
        - days since last matchup
        - last matchup run differential
        - incoming season matchup games
        - incoming season matchup win %

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing game data with team and schedule information. 
    date_col : str, default 'game_date'
        Column name containing date information used for chronological sorting
    date_time_col : str, default 'game_date_time'
        Column name containing datetime information used for chronological sorting. 
        Necessary for sorting between double headers for a single team. 

    Returns
    -------
    pd.DataFrame
        DataFrame with several features at the matchup level.
    """
    # Get schedule information
    df_team_sched = team_schedule(df, date_time_col=date_time_col, date_col=date_col)

    # Days since last matchup game
    df_team_sched['days_since_last_matchup'] = df_team_sched.groupby(['team','opp_team'])['game_date'].diff().dt.days
    df_team_sched['season_start_date'] = pd.to_datetime(df_team_sched['game_date'].dt.year.astype(str) + '-04-01')
    df_team_sched['days_since_season_start_date'] = (df_team_sched['game_date']-df_team_sched['season_start_date']).dt.days
    df_team_sched['days_since_last_matchup'] = df_team_sched['days_since_last_matchup'].fillna(180+df_team_sched['days_since_season_start_date'])
    
    # Last matchup game run differential
    df_team_sched['last_matchup_run_diff'] = df_team_sched.groupby(['team','opp_team'])['team_run_diff'].shift(1)
    df_team_sched['last_matchup_run_diff'] = df_team_sched['last_matchup_run_diff'].fillna(0)

    # Incoming season matchup games and win %
    df_team_sched['season_matchup_wins'] = df_team_sched.groupby(['team','opp_team','season'])['team_win'].cumsum()
    df_team_sched['season_matchup_games'] = df_team_sched.groupby(['team','opp_team','season'])['team_win'].cumcount()+1
    df_team_sched['season_matchup_win%'] = df_team_sched['season_matchup_wins']/df_team_sched['season_matchup_games']
    df_team_sched['incoming_season_matchup_win%'] = df_team_sched.groupby(['team','opp_team','season'])['season_matchup_win%'].shift(1)
    df_team_sched['incoming_season_matchup_win%'] = df_team_sched['incoming_season_matchup_win%'].fillna(0)

    # Drop unecessary columns
    cols_to_keep = ['team','opp_team','game_date','game_date_time','days_since_last_matchup','last_matchup_run_diff','incoming_season_matchup_win%']
    df_team_sched = df_team_sched[cols_to_keep]
    
    return df_team_sched


def merge_team_features_into_games(df_games: pd.DataFrame, df_team_schedule: pd.DataFrame, team_schedule_cols: list[str], date_time_col: str = 'game_date_time'):
    print(f"df_games: {df_games.shape}")
    print(display(df_games.head(3)))
    # Home Merge
    df_games = df_games.merge(
        df_team_schedule[team_schedule_cols + [date_time_col]],
        how='left',
        left_on=['home_team', date_time_col],
        right_on=['team', date_time_col]
    )
    df_games.drop('team', axis=1, inplace=True)
    home_cols_rename = {col: f'home_{col}' for col in team_schedule_cols}
    df_games.rename(columns=home_cols_rename, inplace=True)
    print(f"df_games AFTER Home Merge: {df_games.shape}")
    print(display(df_games.head(3)))

    # Away Merge
    df_games = df_games.merge(
        df_team_schedule[team_schedule_cols + [date_time_col]],
        how='left',
        left_on=['away_team',date_time_col],
        right_on=['team',date_time_col]
    )
    df_games.drop('team', axis=1, inplace=True)
    away_cols_rename = {col: f'away_{col}' for col in team_schedule_cols}
    df_games.rename(columns=away_cols_rename, inplace=True)
    print(f"df_games AFTER Away Merge: {df_games.shape}")
    print(display(df_games.head(3)))

    return df_games

def get_sched_features(df: pd.DataFrame, date_col: str = 'game_date', date_time_col: str = 'game_date_time'):
    # create team schedule data
    df_team_sched = team_schedule(df, date_col=date_col, date_time_col=date_time_col)
    # add team_rest_days
    df_team_sched['team_rest_days'] = team_rest_days(df)
    # add rest_days_7day_avg
    df_team_sched['team_games_prev_7days'] = team_games_previous_7days(df)
    # add matchup features
    df_team_matchup = team_matchup_features(df)
    # concatenate
    df_team_sched = df_team_sched.merge(df_team_matchup, how='left',on=['team','opp_team','game_date','game_date_time'])

    # Merge
    team_sched_cols = ['team','team_rest_days','team_games_prev_7days', 'days_since_last_matchup','last_matchup_run_diff','incoming_season_matchup_win%'] # no date_time_col here -- add separately
    df_games = merge_team_features_into_games(df_games=df, df_team_schedule=df_team_sched,team_schedule_cols=team_sched_cols, date_time_col=date_time_col)

    return df_games

In [9]:
df_ts = team_schedule(df=df,date_col = 'game_date', date_time_col = 'game_date_time')

In [10]:
df_ts.head()

,team,opp_team,game_date,game_date_time,team_score,opp_score,home_ind,team_win,team_run_diff,season
0,Arizona Diamondbacks,San Diego Padres,2021-04-01,2021-04-01 20:10:00+00:00,7.0,8.0,0,0,-1.0,2021
1,Arizona Diamondbacks,San Diego Padres,2021-04-02,2021-04-03 02:10:00+00:00,2.0,4.0,0,0,-2.0,2021
2,Arizona Diamondbacks,San Diego Padres,2021-04-03,2021-04-04 00:40:00+00:00,0.0,7.0,0,0,-7.0,2021
3,Arizona Diamondbacks,San Diego Padres,2021-04-04,2021-04-04 20:10:00+00:00,3.0,1.0,0,1,2.0,2021
4,Arizona Diamondbacks,Colorado Rockies,2021-04-06,2021-04-07 00:40:00+00:00,10.0,8.0,0,1,2.0,2021


In [9]:
df.head()

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2.0,3.0,Final,Yankee Stadium,R
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Guardians,3.0,2.0,Final,Comerica Park,R
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,158,142,Milwaukee Brewers,Minnesota Twins,6.0,5.0,Final,American Family Field,R
3,634634,2021-04-01,2021-04-01 18:20:00+00:00,112,134,Chicago Cubs,Pittsburgh Pirates,3.0,5.0,Final,Wrigley Field,R
4,634622,2021-04-01,2021-04-01 19:05:00+00:00,143,144,Philadelphia Phillies,Atlanta Braves,3.0,2.0,Final,Citizens Bank Park,R


In [12]:
df_sched = get_sched_features(df)

df_games: (11041, 12)


,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2.0,3.0,Final,Yankee Stadium,R
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Guardians,3.0,2.0,Final,Comerica Park,R
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,158,142,Milwaukee Brewers,Minnesota Twins,6.0,5.0,Final,American Family Field,R


None
df_games AFTER Home Merge: (11041, 17)


,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,home_team_rest_days,home_team_games_prev_7days,home_days_since_last_matchup,home_last_matchup_run_diff,home_incoming_season_matchup_win%
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2.0,3.0,Final,Yankee Stadium,R,0.0,0.0,180.0,0.0,0.0
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Guardians,3.0,2.0,Final,Comerica Park,R,0.0,0.0,180.0,0.0,0.0
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,158,142,Milwaukee Brewers,Minnesota Twins,6.0,5.0,Final,American Family Field,R,0.0,0.0,180.0,0.0,0.0


None
df_games AFTER Away Merge: (11041, 22)


,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,home_team_rest_days,home_team_games_prev_7days,home_days_since_last_matchup,home_last_matchup_run_diff,home_incoming_season_matchup_win%,away_team_rest_days,away_team_games_prev_7days,away_days_since_last_matchup,away_last_matchup_run_diff,away_incoming_season_matchup_win%
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2.0,3.0,Final,Yankee Stadium,R,0.0,0.0,180.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Guardians,3.0,2.0,Final,Comerica Park,R,0.0,0.0,180.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,158,142,Milwaukee Brewers,Minnesota Twins,6.0,5.0,Final,American Family Field,R,0.0,0.0,180.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0


None


In [17]:
df_ts['days_since_last_matchup'] = df_ts.groupby(['team','opp_team'])['game_date'].diff().dt.days
df_ts['season_start_date'] = pd.to_datetime(df_ts['game_date'].dt.year.astype(str) + '-04-01')
df_ts['days_since_season_start_date'] = (df_ts['game_date']-df_ts['season_start_date']).dt.days
df_ts['days_since_last_matchup'] = df_ts['days_since_last_matchup'].fillna(180+df_ts['days_since_season_start_date'])
df_ts['matchup_num'] = df_ts.groupby(['team','opp_team']).cumcount()+1
df_ts['last_matchup_run_diff'] = df_ts.groupby(['team','opp_team'])['team_run_diff'].shift(1)
df_ts['last_matchup_run_diff'] = df_ts['last_matchup_run_diff'].fillna(0)
df_ts['season_matchup_wins'] = df_ts.groupby(['team','opp_team','season'])['team_win'].cumsum()
df_ts['season_matchup_games'] = df_ts.groupby(['team','opp_team','season'])['team_win'].cumcount()+1
df_ts['season_matchup_win%'] = df_ts['season_matchup_wins']/df_ts['season_matchup_games']
df_ts['incoming_season_matchup_win%'] = df_ts.groupby(['team','opp_team','season'])['season_matchup_win%'].shift(1)
df_ts['incoming_season_matchup_win%'] = df_ts['incoming_season_matchup_win%'].fillna(0)

df_ts.drop(['season_start_date','days_since_season_start_date','team_score','opp_score','team_run_diff'], axis=1, inplace=True)

In [18]:
df_ts.head()

,team,opp_team,game_date,game_date_time,home_ind,team_win,season,days_since_last_matchup,matchup_num,last_matchup_run_diff,season_matchup_wins,season_matchup_games,season_matchup_win%,incoming_season_matchup_win%
0,Arizona Diamondbacks,San Diego Padres,2021-04-01,2021-04-01 20:10:00+00:00,0,0,2021,180.0,1,0.0,0,1,0.00,NaN
1,Arizona Diamondbacks,San Diego Padres,2021-04-02,2021-04-03 02:10:00+00:00,0,0,2021,1.0,2,-1.0,0,2,0.00,0.0
2,Arizona Diamondbacks,San Diego Padres,2021-04-03,2021-04-04 00:40:00+00:00,0,0,2021,1.0,3,-2.0,0,3,0.00,0.0
3,Arizona Diamondbacks,San Diego Padres,2021-04-04,2021-04-04 20:10:00+00:00,0,1,2021,1.0,4,-7.0,1,4,0.25,0.0
4,Arizona Diamondbacks,Colorado Rockies,2021-04-06,2021-04-07 00:40:00+00:00,0,1,2021,185.0,1,0.0,1,1,1.00,NaN


In [19]:
df_ts[(df_ts['team']=='Los Angeles Dodgers') & (df_ts['opp_team']=='Seattle Mariners')]

,team,opp_team,game_date,game_date_time,home_ind,team_win,season,days_since_last_matchup,matchup_num,last_matchup_run_diff,season_matchup_wins,season_matchup_games,season_matchup_win%,incoming_season_matchup_win%
9555,Los Angeles Dodgers,Seattle Mariners,2021-04-19,2021-04-20 02:10:00+00:00,0,0,2021,198.0,1,0.0,0,1,0.000000,NaN
9556,Los Angeles Dodgers,Seattle Mariners,2021-04-20,2021-04-20 20:10:00+00:00,0,1,2021,1.0,2,-1.0,1,2,0.500000,0.000000
9573,Los Angeles Dodgers,Seattle Mariners,2021-05-11,2021-05-12 02:10:00+00:00,1,1,2021,21.0,3,1.0,2,3,0.666667,0.500000
9574,Los Angeles Dodgers,Seattle Mariners,2021-05-12,2021-05-13 02:10:00+00:00,1,1,2021,1.0,4,2.0,3,4,0.750000,0.666667
10006,Los Angeles Dodgers,Seattle Mariners,2023-09-15,2023-09-16 02:10:00+00:00,0,1,2023,856.0,5,6.0,1,1,1.000000,NaN
10007,Los Angeles Dodgers,Seattle Mariners,2023-09-16,2023-09-17 01:40:00+00:00,0,1,2023,1.0,6,3.0,2,2,1.000000,1.000000
10008,Los Angeles Dodgers,Seattle Mariners,2023-09-17,2023-09-17 20:10:00+00:00,0,1,2023,1.0,7,4.0,3,3,1.000000,1.000000
10147,Los Angeles Dodgers,Seattle Mariners,2024-08-19,2024-08-20 02:10:00+00:00,1,1,2024,337.0,8,5.0,1,1,1.000000,NaN
10148,Los Angeles Dodgers,Seattle Mariners,2024-08-20,2024-08-21 02:10:00+00:00,1,1,2024,1.0,9,3.0,2,2,1.000000,1.000000
10149,Los Angeles Dodgers,Seattle Mariners,2024-08-21,2024-08-22 02:10:00+00:00,1,1,2024,1.0,10,3.0,3,3,1.000000,1.000000


In [20]:
df_ts[df_ts['days_since_last_matchup']==3]

,team,opp_team,game_date,game_date_time,home_ind,team_win,season,days_since_last_matchup,matchup_num,last_matchup_run_diff,season_matchup_wins,season_matchup_games,season_matchup_win%,incoming_season_matchup_win%
1090,Atlanta Braves,New York Mets,2023-05-01,2023-05-01 17:10:00+00:00,0,1,2023,3.0,38,4.0,2,2,1.000,1.000000
2202,Baltimore Orioles,Cleveland Guardians,2025-07-24,2025-07-24 17:10:00+00:00,0,1,2025,3.0,32,-5.0,3,5,0.600,0.500000
3686,Chicago White Sox,Kansas City Royals,2021-04-11,2021-04-11 18:10:00+00:00,1,0,2021,3.0,2,6.0,1,2,0.500,1.000000
5853,Cleveland Guardians,Baltimore Orioles,2025-07-24,2025-07-24 17:10:00+00:00,1,0,2025,3.0,32,5.0,2,5,0.400,0.500000
8065,Kansas City Royals,Chicago White Sox,2021-04-11,2021-04-11 18:10:00+00:00,0,1,2021,3.0,2,-6.0,1,2,0.500,0.000000
12673,New York Mets,Philadelphia Phillies,2022-05-08,2022-05-08 16:35:00+00:00,0,0,2022,3.0,25,1.0,5,8,0.625,0.714286
12826,New York Mets,Atlanta Braves,2023-05-01,2023-05-01 17:10:00+00:00,1,0,2023,3.0,38,-4.0,0,2,0.000,0.000000
14870,Philadelphia Phillies,New York Mets,2022-05-08,2022-05-08 16:35:00+00:00,1,1,2022,3.0,25,-1.0,3,8,0.375,0.285714


In [21]:
df_ts[(df_ts['team']=='Atlanta Braves') & (df_ts['opp_team']=='New York Mets')]

,team,opp_team,game_date,game_date_time,home_ind,team_win,season,days_since_last_matchup,matchup_num,last_matchup_run_diff,season_matchup_wins,season_matchup_games,season_matchup_win%,incoming_season_matchup_win%
784,Atlanta Braves,New York Mets,2021-05-17,2021-05-17 23:10:00+00:00,1,0,2021,226.0,1,0.0,0,1,0.000000,NaN
785,Atlanta Braves,New York Mets,2021-05-18,2021-05-18 23:20:00+00:00,1,0,2021,1.0,2,-2.0,0,2,0.000000,0.000000
786,Atlanta Braves,New York Mets,2021-05-19,2021-05-19 23:20:00+00:00,1,1,2021,1.0,3,-1.0,1,3,0.333333,0.000000
793,Atlanta Braves,New York Mets,2021-05-29,2021-05-29 23:15:00+00:00,0,0,2021,10.0,4,1.0,1,4,0.250000,0.333333
812,Atlanta Braves,New York Mets,2021-06-21,2021-06-21 21:10:00+00:00,0,0,2021,23.0,5,-11.0,1,5,0.200000,0.250000
813,Atlanta Braves,New York Mets,2021-06-22,2021-06-22 23:10:00+00:00,0,1,2021,1.0,6,-2.0,2,6,0.333333,0.200000
814,Atlanta Braves,New York Mets,2021-06-23,2021-06-23 23:10:00+00:00,0,0,2021,1.0,7,3.0,2,7,0.285714,0.333333
819,Atlanta Braves,New York Mets,2021-06-29,2021-06-29 23:20:00+00:00,1,0,2021,6.0,8,-4.0,2,8,0.250000,0.285714
820,Atlanta Braves,New York Mets,2021-06-30,2021-06-30 23:20:00+00:00,1,1,2021,1.0,9,-1.0,3,9,0.333333,0.250000
821,Atlanta Braves,New York Mets,2021-07-01,2021-07-01 23:20:00+00:00,1,1,2021,1.0,10,18.0,4,10,0.400000,0.333333
